In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# 데이터 불러오기 
train  = pd.read_csv('news_data/train.csv')
test = pd.read_csv('news_data/test.csv')

train.head()

,id,text,target
0,0,"\nThey were, and even if Washington might cons...",10
1,1,"We run ""SpaceNews & Views"" on our STAREACH BBS...",14
2,2,\n\n\nNot to worry. The Masons have been demo...,19
3,3,"Only Brendan McKay, or maybe ARF, would come t...",17
4,4,Help: I am running some sample problems from O...,5


In [3]:
train.shape

(9233, 3)

In [4]:
# 결측치 확인 
def check_missing_col(dataframe):
    missing_col = []
    for col in dataframe.columns:
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            print(f'결측치가 있는 컬럼은: {col} 입니다')
            print(f'해당 컬럼에 총 {missing_values} 개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if missing_col == []:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(train)

결측치가 존재하지 않습니다


In [8]:
X = train.text # 문서 추출
y = train.target # 라벨 추출

In [6]:
X.head()

0    \nThey were, and even if Washington might cons...
1    We run "SpaceNews & Views" on our STAREACH BBS...
2    \n\n\nNot to worry.  The Masons have been demo...
3    Only Brendan McKay, or maybe ARF, would come t...
4    Help: I am running some sample problems from O...
Name: text, dtype: object

In [9]:
y.head()

0    10
1    14
2    19
3    17
4     5
Name: target, dtype: int64

## CountVectorizer 

컴퓨터는 일반적으로 문장(텍스트)을 계산할 수 없고 숫자형 피처를 데이터로 입력받아 동작하기 때문에

텍스트와 같은 데이터는 머신러닝 알고리즘에 바로 입력할 수가 없습니다.

따라서 컴퓨터가 계산을 하기 위해서는 먼저 문장을 숫자형 값으로 바꾼 다음 계산하도록 해야 합니다.

이것을 워드 임베딩(word embedding)이라고 합니다.

이번 베이스라인에서는 CountVectorizer 기법을 사용해 문장을 숫자로 표현해보겠습니다.

CountVectorizer 는 입력된 문장을 토큰화(Tokenize)하여 토큰의 등장 빈도 벡터로 바꿔주는 기법입니다.

토큰화란 문장를 의미가 있는 최소의 단위로 쪼개는 것을 의미합니다.

예를 들어 "hello, I am a data scientist!" 라는 문장을 띄어쓰기 기준으로 토큰화 시키면

"hello", ",", "I", "am", "a", "data", "scientist", "!" 라는 8개의 토큰으로 토큰화 됩니다.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

sample_vectorizer = CountVectorizer()

어떠한 단어들을 사용할 지, 어떠한 단어가 중요도를 가지는지 학습하는 과정입니다.

CountVectorizer 의 경우 단어의 출현 빈도를 기준으로 문장을 벡터(=숫자의 집합)로 바꿔줍니다.

In [12]:
s_text1 = ['hellow, I am won then wish to a data scientist']

sample_vectorizer.fit(s_text1)

CountVectorizer()

In [13]:
print(sample_vectorizer.vocabulary_)

{'hellow': 2, 'am': 0, 'won': 7, 'then': 4, 'wish': 6, 'to': 5, 'data': 1, 'scientist': 3}


In [16]:
# data가 두번 등장하여 2 , 나머지 0 
s_text2 = ['learning data analysis and data science ']

s_vector = sample_vectorizer.transform(s_text2)
print(s_vector.toarray())

[[0 2 0 0 0 0 0 0]]


단어들의 출현 빈도로 이루어진 크기 8의 백터가 출력되었습니다. 이 벡터를 BoW 라고 합니다.

BoW 란 "Bag of Words" 의 약자로 CountVectorizer 로 변환된 단어의 집합을 말합니다

특정 단어가 N번 등장했다면, 그 특정 단어가 N개가 들어있습니다.

또한 단어를 섞었기 때문에 더 이상 단어의 순서는 중요하지 않습니다.

BoW를 만드는 과정은 이렇게 두 가지 과정이 있습니다.

* 우선 각 단어에 고유한 정수 인덱스를 부여합니다.

* 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만듭니다.

BoW는 각 단어가 등장한 횟수를 수치화하는 텍스트 표현 방법이기 때문에,

주로 어떤 단어가 얼마나 등장했는지를 기준으로 문서가 어떤 성격의 문서인지를 판단하는 작업에 쓰입니다.

즉, 분류 문제나 여러 문서 간의 유사도를 구하는 문제에 주로 쓰입니다.

## 뉴스 그룹 데이터

In [17]:
v = CountVectorizer()
v.fit(X)
X = v.transform(X)

In [18]:
v.inverse_transform(X[0])#역변환하여 첫번째 문장의 단어들 확인

[array(['and', 'been', 'bust', 'complete', 'consider', 'druce', 'even',
        'goals', 'has', 'here', 'if', 'in', 'might', 'minute', 'only',
        'patty', 'rework', 'that', 'they', 'trade', 'utter', 'washington',
        'were'], dtype='<U81')]

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 


model = LogisticRegression(max_iter=500) #객체에 모델 할당
model.fit(X, y) #모델 학습

LogisticRegression(max_iter=500)

In [22]:
from sklearn.metrics import accuracy_score

#run model
y_pred = model.predict(X[0])
print('예측 라벨 : ', y_pred)
print('실제 라벨 : ', train.target[0])

예측 라벨 :  [10]
실제 라벨 :  10
